In [1]:
# 라이브러리
import sys
import os
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from selenium import webdriver
import time
from tqdm import tqdm_notebook

In [2]:
# 테스트 데이터 로드
y_reple = pd.read_excel('./대전광역시_관광명소_202101082.xlsx')

In [8]:
y_reple.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60 entries, 0 to 59
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  60 non-null     int64 
 1   구분          60 non-null     object
 2   명소명         60 non-null     object
 3   소재지         60 non-null     object
 4   전화번호        57 non-null     object
 5   상세소개        60 non-null     object
 6   담당부서        60 non-null     object
 7   전화부서        60 non-null     object
dtypes: int64(1), object(7)
memory usage: 3.9+ KB


In [3]:
# 불량 데이터 처리
y_reple.iloc[35,3]=' '
y_reple.iloc[5,3]=y_reple.iloc[5,4]
y_reple.iloc[17,3]="대전광역시 유성구 장대동 191"
y_reple.iloc[20,3]='대전광역시 유성구 원촌동 산35-1'
y_reple.iloc[23,3]='대전 대덕구 쌍청당로 25 대덕구청소년수련관'

In [5]:
y_reple.head()

,Unnamed: 0,구분,명소명,소재지,전화번호,상세소개,담당부서,전화부서
0,0,역사명소,계족산성,대전광역시 대덕구 장동 산85,042-623-9909,https://www.daejeon.go.kr/tou/gallery40BoardVi...,관광마케팅과,042-270-3984
1,1,역사명소,미륵원,대전광역시 동구 마산동 135-2,042-251-4204,https://www.daejeon.go.kr/tou/gallery40BoardVi...,관광마케팅과,042-270-3984
2,2,역사명소,동춘당,대전광역시 대덕구 동춘당로 80 (송촌동),042-608-6572,https://www.daejeon.go.kr/tou/gallery40BoardVi...,관광마케팅과,042-270-3984
3,3,역사명소,남간정사,대전광역시 동구 충정로 53 (가양동),042-673-9286,https://www.daejeon.go.kr/tou/gallery40BoardVi...,관광마케팅과,042-270-3984
4,4,역사명소,단재 신채호 선생 생가터,대전광역시 중구 단재로229번길 47,042-606-6283,https://www.daejeon.go.kr/tou/gallery40BoardVi...,관광마케팅과,042-270-3984


In [11]:
y_reple.fillna(' ')

,Unnamed: 0,구분,명소명,소재지,전화번호,상세소개,담당부서,전화부서
0,0,역사명소,계족산성,대전광역시 대덕구 장동 산85,042-623-9909,https://www.daejeon.go.kr/tou/gallery40BoardVi...,관광마케팅과,042-270-3984
1,1,역사명소,미륵원,대전광역시 동구 마산동 135-2,042-251-4204,https://www.daejeon.go.kr/tou/gallery40BoardVi...,관광마케팅과,042-270-3984
2,2,역사명소,동춘당,대전광역시 대덕구 동춘당로 80 (송촌동),042-608-6572,https://www.daejeon.go.kr/tou/gallery40BoardVi...,관광마케팅과,042-270-3984
3,3,역사명소,남간정사,대전광역시 동구 충정로 53 (가양동),042-673-9286,https://www.daejeon.go.kr/tou/gallery40BoardVi...,관광마케팅과,042-270-3984
4,4,역사명소,단재 신채호 선생 생가터,대전광역시 중구 단재로229번길 47,042-606-6283,https://www.daejeon.go.kr/tou/gallery40BoardVi...,관광마케팅과,042-270-3984


In [27]:
# 위도 경도를 구하기 위하여 주소를 따로 저장
address=y_reple.iloc[:,3]

In [32]:
list1=[]
# 위도 경도 구하는 사이트 들어가기
path = "chromedriver.exe"
driver = webdriver.Chrome(path)
driver.get('http://www.dawuljuso.com/')
time.sleep(2)
# len(address) 원하는 주소를 다구하기 위해서
for i in range(0,len(address)):
    # 검색창에 키 값 입력
    element = driver.find_element_by_id("input_juso")
    element.send_keys(address[i])
    time.sleep(1)
    # 실행
    driver.find_elements_by_xpath("./html/body/div[4]/div[1]/div[1]/div[2] ")[0].click( )
    time.sleep(1)
    # 위도경도를 저장 append로 저장
    t=driver.find_elements_by_xpath('.//*[@id="insert_data_5"]')[0].text
    list1.append(t)

In [33]:
# 데이터 프레임화
t_data=pd.DataFrame({'위도,경도':list1})

In [39]:
# 데이터 정제
latitude=[]
longitude=[]

for i in range(0,len(t_data.iloc[:,0])):
   
    tt=t_data.iloc[i,0].split(',')    # 쪼개고
    
    q=tt[0].split(':')               # 또 쪼개고(위도)
    latitude.append(q[1])
    
    w=tt[1].split(':')               # 또또 쪼개고(경도)
    longitude.append(w[1])

In [41]:
# 데이터 프레임화

result_df=pd.DataFrame({'명소명':list(y_reple.iloc[:,2]),
                        "latitude":latitude,
                        "longitude":longitude,
                       })

In [42]:
result_df

,명소명,latitude,longitude
0,계족산성,127.453847,36.3947025
1,미륵원,127.491920,36.3869194
2,동춘당,127.441454,36.3652808
3,남간정사,127.459215,36.3481910
4,단재 신채호 선생 생가터,127.411088,36.2327086
5,"""옛 대전형무소 망루",127.410787,36.3369579
6,수운교의 성지 천단,127.342140,36.4210719
7,이사동 민속마을,127.449691,36.2880649
8,옛 충남도청과 근현대사전시관,127.420912,36.3266855
9,대전역과 추억의 가락국수,127.431056,36.3304975


In [43]:
result_df.to_excel("result_lat_long.xlsx", encoding='utf-8-sig')